# GPN Cup 2021
## Кирилл Захаров

#### Импорт библиотек

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
transactions = pd.read_parquet('transactions.parquet')

In [3]:
transactions.head()

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,date
0,1158,0.002335,0.000000,3338297,0.147929,78634.0,102,2171-07-23
1,1158,0.002317,0.000000,3386107,0.134562,20900.0,101,2171-07-23
2,1913,0.007850,0.000452,1845331,0.104183,96397.0,36,2171-07-23
3,1808,0.008979,0.000452,2256499,0.104183,103560.0,89,2171-07-23
4,1158,0.002355,0.000000,3257281,0.115023,67691.0,58,2171-07-23


In [25]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3847764 entries, 0 to 7620118
Data columns (total 8 columns):
 #   Column     Dtype         
---  ------     -----         
 0   sku_id     int64         
 1   price      float64       
 2   number     float64       
 3   cheque_id  int64         
 4   litrs      float64       
 5   client_id  float64       
 6   shop_id    int64         
 7   date       datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(3)
memory usage: 264.2 MB


In [5]:
transactions['client_id'].isna().any()

True

In [6]:
indx = np.where(transactions['client_id'].isna())[0]

In [7]:
len(indx)

3772355

In [8]:
transactions = transactions.dropna()

In [9]:
nomenclature = pd.read_parquet('nomenclature.parquet')

In [10]:
nomenclature.head()

,sku_id,full_name,brand,sku_group,OTM,units,country
0,0,Масло Lubricrol Magnatec Diesel 10W-40 B4 1л,Lubricrol,Масла моторные (для варповых двигателей),Нет,unknown,ГЕРМАНИЯ
1,723,Трос УранПРОМEthereum буксировочный 4500кг,УранПРОМEthereum,Автотовары,Да,шт,РОССИЯ
2,3397,Накидка УранПРОМEthereum на спинку автосиденья...,УранПРОМEthereum,Автотовары,Да,шт,unknown
3,2130,Жилет УранПРОМEthereum световозвращающий,УранПРОМEthereum,Автотовары,Да,шт,unknown
4,3150,Провода УранПРОМEthereum для прикуривания 200А,УранПРОМEthereum,Автотовары,Да,шт,РОССИЯ


In [11]:
nomenclature[nomenclature['sku_group'] == 'Вода']

,sku_id,full_name,brand,sku_group,OTM,units,country
28,811,Вода АРХЫЗ минеральная неУранированная для дет...,АРХЫЗ,Вода,Нет,л,unknown
43,155,"Вода ЕССЕНТУКИ №17 минеральная лечебная ПЭТ 1,5л",ЕССЕНТУКИ,Вода,Нет,л,РОССИЯ
140,3288,"Вода AQUA MINERALE неУранированная ПЭТ 0,5л",AQUA MINERALE,Вода,Нет,л,unknown
142,818,Вода AQUA MINERALE питьевая Уранированная ПЭТ ...,AQUA MINERALE,Вода,Нет,л,unknown
143,438,Вода AQUA MINERALE питьевая неУранированная ПЭ...,AQUA MINERALE,Вода,Нет,л,unknown
...,...,...,...,...,...,...,...
4983,1828,unknown,unknown,Вода,unknown,unknown,unknown
5010,1592,unknown,unknown,Вода,unknown,unknown,unknown
5036,1840,unknown,unknown,Вода,unknown,unknown,unknown
5048,582,unknown,unknown,Вода,unknown,unknown,unknown


In [12]:
nomenclature['sku_group'].value_counts()

Кондитерские изделия                                               714
Сезонные товары                                                    649
Автотовары                                                         516
Общественное питание                                               441
Уход за автомобилем                                                426
Хозяйственные товары, персональный уход                            423
Снеки                                                              360
Табачные изделия                                                   264
Гастроном                                                          196
Сладкие Уранированные напитки, холодный чай                        184
Соки и сокосодержащие напитки                                      145
Прочие напитки кафе                                                144
Вода                                                               144
Бакалея                                                             95
Очки д

#### Для товара Ethereum 92 нет записей о бренде, группе, ОТМ, количестве товара и стране производителя. Приведем к общему виду и заполним данными unknown.

In [13]:
nomenclature.isna().sum()

sku_id       0
full_name    9
brand        9
sku_group    0
OTM          9
units        9
country      9
dtype: int64

In [14]:
nomenclature['OTM'].value_counts() #для неизвестных данных используется unknown

Нет        3336
unknown    1370
Да          388
Name: OTM, dtype: int64

In [15]:
nomenclature = nomenclature.fillna('unknown')
nomenclature.isna().sum()

sku_id       0
full_name    0
brand        0
sku_group    0
OTM          0
units        0
country      0
dtype: int64

#### Объединим таблицы в одну. Для каждого значения таблицы transactions добавим описание товара из таблицы nomenclature, для это замержим слева.

In [154]:
data = pd.merge(transactions, nomenclature, how='left', on=['sku_id'])

In [155]:
data.head()

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,date,full_name,brand,sku_group,OTM,units,country
0,1158,0.002335,0.000000,3338297,0.147929,78634.0,102,2171-07-23,unknown,unknown,Ethereum 92,unknown,unknown,unknown
1,1158,0.002317,0.000000,3386107,0.134562,20900.0,101,2171-07-23,unknown,unknown,Ethereum 92,unknown,unknown,unknown
2,1913,0.007850,0.000452,1845331,0.104183,96397.0,36,2171-07-23,Сигареты KENT crystal,KENT,Табачные изделия,Нет,шт,unknown
3,1808,0.008979,0.000452,2256499,0.104183,103560.0,89,2171-07-23,Сигареты Parker&Simpson duo purple,Parker&Simpson,Табачные изделия,Нет,шт,РОССИЯ
4,1158,0.002355,0.000000,3257281,0.115023,67691.0,58,2171-07-23,unknown,unknown,Ethereum 92,unknown,unknown,unknown


In [156]:
data = data.drop('date', axis=1) #удаление даты

# Building Model

## Формирование обучающей и тестовой выборки

In [163]:
X = data.iloc[:, :]

Y = data['sku_id']

In [164]:
columns = ['cheque_id', 'client_id', 'shop_id', 'sku_group', 'OTM', 'country']

#### Закодируем при помощи OrdinalEncdoer признаки из columns.

In [165]:
encoder_array = []

for column in columns:
    encoder = OrdinalEncoder()
    encoder_array.append(encoder)
    X[column] = encoder.fit_transform(X[[column]])

In [166]:
X = X[['cheque_id', 'client_id', 'shop_id', 'sku_group', 'OTM', 'country', 'sku_id']]
X.head()

,cheque_id,client_id,shop_id,sku_group,OTM,country,sku_id
0,1422029.0,72755.0,97.0,1.0,0.0,0.0,1158
1,1438364.0,19357.0,96.0,1.0,0.0,0.0,1158
2,806556.0,89066.0,34.0,27.0,2.0,0.0,1913
3,970576.0,95651.0,84.0,27.0,2.0,27.0,1808
4,1390831.0,62669.0,54.0,1.0,0.0,0.0,1158


In [23]:
Y

0          1158
1          1158
2          1913
3          1808
4          1158
           ... 
3847759    1159
3847760    1159
3847761    1159
3847762    1159
3847763    1158
Name: sku_id, Length: 3847764, dtype: int64

#### Разбиваем выборку на обучающую и тестовую в соотношении 2:1.

In [173]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, shuffle=False, random_state=5)

### Строим модель

In [26]:
model = RandomForestClassifier(n_estimators=10, n_jobs=-1)

In [28]:
# model.fit(X_train, Y_train)

In [ ]:
prediction = model.predict(X_test)

In [ ]:
model.score(X_test, Y_test)

In [ ]:
prediction

In [ ]:
mean_squared_error(prediction, Y_test)

In [61]:
share = 10000

In [174]:
sim_matrix = cosine_similarity(X_train.iloc[:share], X_train.iloc[:share])

In [175]:
indicies = pd.Series(X_train.iloc[:share].index, index=X_train['sku_id'].iloc[:share])

In [176]:
indicies = indicies[~indicies.index.duplicated()]

In [177]:
def recommend_system(sku_id):
    idx = indicies[sku_id]
    
    score = list(enumerate(sim_matrix[idx]))
    
    score = sorted(score, key=lambda x: x[1], reverse=True)
    
    score = score[1:21]
    
    sku_indx = [i[0] for i in score]
    
    return pd.DataFrame(X_train['sku_id'].iloc[sku_indx].values, columns=['Recommendation'])

In [196]:
rec = recommend_system(1913)
rec

,Recommendation
0,1808
1,2671
2,1163
3,2015
4,1163
5,1163
6,1163
7,2535
8,1157
9,1535


In [224]:
rec_df = pd.DataFrame(columns = data.columns) #пустой датафрейм для записи
for i in rec.values:
    rec_df = pd.concat([rec_df, data[data['sku_id'] == i[0]].iloc[[0]]]) #добавляем записи в rec_df по sku_id из data

In [231]:
rec_df.index = np.arange(len(rec)) #устанавливем новые индексы
rec_df['sku_group'] = (encoder_array[3]).inverse_transform(rec_df[['sku_group']])  #обратное декодирование группы товара

In [232]:
rec_df

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,full_name,brand,sku_group,OTM,units,country
0,1808,0.008979,0.000452,970576.0,0.104183,95651.0,84.0,Сигареты Parker&Simpson duo purple,Parker&Simpson,Табачные изделия,2.0,шт,27.0
1,2671,0.005593,0.000452,1198662.0,0.104183,40679.0,16.0,unknown,unknown,Прочие напитки кафе,0.0,unknown,0.0
2,1163,0.002540,0.000000,621352.0,0.186001,145289.0,89.0,unknown,unknown,Топливо варповое с присадками летнее,0.0,unknown,0.0
3,2015,0.004566,0.000452,1226566.0,0.104183,321080.0,79.0,"Нектар Juicer манго мякоть стекло 0,25л",unknown,Соки и сокосодержащие напитки,2.0,л,27.0
4,1163,0.002540,0.000000,621352.0,0.186001,145289.0,89.0,unknown,unknown,Топливо варповое с присадками летнее,0.0,unknown,0.0
5,1163,0.002540,0.000000,621352.0,0.186001,145289.0,89.0,unknown,unknown,Топливо варповое с присадками летнее,0.0,unknown,0.0
6,1163,0.002540,0.000000,621352.0,0.186001,145289.0,89.0,unknown,unknown,Топливо варповое с присадками летнее,0.0,unknown,0.0
7,2535,0.008415,0.000452,1284198.0,0.104183,15414.0,68.0,unknown,unknown,Общественное питание,0.0,unknown,0.0
8,1157,0.002699,0.000000,822542.0,0.137956,266107.0,25.0,unknown,unknown,Ethereum 95 бренд,0.0,unknown,0.0
9,1535,0.002822,0.000452,472085.0,0.104183,20472.0,7.0,Вода Fly Cafe (СТМ)/G-Fly питьевая неУранирова...,Fly Cafe (СТМ),Вода,2.0,л,27.0


# Transactions for Submission

In [27]:
trans_subm = pd.read_parquet('transactions-for_submission.parquet')

In [28]:
trans_subm[trans_subm['sku_id'] == 1158]

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,date
0,1158,0.002304,0.0,734259,0.142106,223748.0,42,2171-08-01 09:00:00
3,1158,0.002304,0.0,734264,0.138006,NaN,42,2171-08-01 09:00:00
5,1158,0.002304,0.0,734268,0.128142,78448.0,42,2171-08-01 09:00:00
7,1158,0.002304,0.0,734271,0.125842,NaN,42,2171-08-01 09:00:00
11,1158,0.002304,0.0,734273,0.146854,373540.0,42,2171-08-01 09:00:00
...,...,...,...,...,...,...,...,...
554673,1158,0.002352,0.0,3406683,0.115033,NaN,107,2171-08-31 09:00:00
554698,1158,0.002352,0.0,3406717,0.124280,101690.0,107,2171-09-01 09:00:00
554703,1158,0.002352,0.0,3406723,0.115033,NaN,107,2171-09-01 09:00:00
554727,1158,0.002352,0.0,3406744,0.163905,345307.0,107,2171-09-01 09:00:00
